### 1) Загрузка таблиц по ссылкам и объединение ###

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display

url_1 = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%93%D0%9D%D0%A4_%D0%91_%D0%9D_000006989.html"
url_2 = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%93%D0%9D%D0%A4_%D0%91_%D0%98_000007463.html"


def read_table(url: str) -> pd.DataFrame:
    tables = pd.read_html(url, header=0, skiprows=12)
    df = tables[0].copy()
    return df

# Загружаем обе таблицы и объединяем
top = read_table(url_1)
bottom = read_table(url_2)
raw_df = pd.concat([top, bottom], ignore_index=True, sort=False)
display(raw_df)


### 2) Приведение к "рабочему" виду ###

In [ ]:
df = raw_df.copy()

# Заменяем сложные названия предметов на простые
df.columns = [str(c).strip().replace("\n", " ") for c in df.columns]
rename_map = {}
for c in df.columns:
    if ('АиНМА' in c) or ('Математика' in c):
        rename_map[c] = 'Предмет №1'
    elif any(k in c for k in ['ПФ', 'География', 'Информатика и ИКТ', 'Физика', 'Химия']):
        rename_map[c] = 'Предмет №2'
    elif 'Русский' in c:
        rename_map[c] = 'Русский язык'
df = df.rename(columns=rename_map)

# Удаляем полностью пустые столбцы
df = df.dropna(axis=1, how='all')

# Нормализуем заголовки
df.columns = [str(c).strip().replace('\n', ' ') for c in df.columns]


# Удаляем служебные столбцы, если присутствуют
cols_to_drop = [
    '№',
    'Уникальный код',
    'Представление приказа',
    'Учебная группа',
    'Идентификационный номер заказчика целевого обучения (для целевого приема)',
    'Номер предложения (для целевого приема)'
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Переустанавливаем индексацию
df = df.reset_index(drop=True)
df.index = df.index + 1
df.index.name = '№'

display(df.head(15))


### 3) Проверка типов и приведение количественных к числовому типу ###

In [ ]:
print("До преобразования:")
print(df.dtypes)

numeric_candidates = [
    c for c in df.columns
    if any(k in c for k in ['Сумма баллов', 'Предмет №1', 'Предмет №2', 'Русский'])
]

for c in numeric_candidates:
    df[c] = pd.to_numeric(
        df[c].astype(str).str.replace(',', '.', regex=False),
        errors='coerce'
    )

print("\nПосле преобразования:")
print(df[numeric_candidates].dtypes)

display(df)


### 4) Средний, минимальный и максимальный суммарные баллы ###

In [ ]:
if 'Сумма баллов' not in df.columns:
    raise ValueError("Колонка 'Сумма баллов' не найдена.")

mean_total = df['Сумма баллов'].mean()
min_total = df['Сумма баллов'].min()
max_total = df['Сумма баллов'].max()

print(f"Средняя сумма баллов: {mean_total:.2f}")
print("Минимальная сумма баллов:", min_total)
print("Максимальная сумма баллов:", max_total)
display(df)


### 5) Средние по трём предметам и предмет с наибольшим средним ###

In [ ]:
required_cols = ['Предмет №1', 'Предмет №2', 'Русский язык']
missing = [c for c in required_cols if c not in df.columns]

means_df = pd.DataFrame({
    'Предмет №1': [df['Предмет №1'].mean()],
    'Предмет №2': [df['Предмет №2'].mean()],
    'Русский язык': [df['Русский язык'].mean()],
}, index=['Среднее'])

display(means_df)

top_subject = means_df.idxmax(axis=1).values[0]
print("Предмет с наибольшим средним баллом:", top_subject)


### 6) Количество абитуриентов Тип_1 ###

In [ ]:
candidates = {c: c for c in df.columns if c in ['Русский язык', 'Предмет №1', 'Предмет №2']}
rus_col = candidates.get('Русский язык')
subj1_col = candidates.get('Предмет №1')
subj2_col = candidates.get('Предмет №2')

rus_mean = df[rus_col].mean() if rus_col else np.nan
subj1_mean = df[subj1_col].mean() if subj1_col else np.nan
subj2_mean = df[subj2_col].mean() if subj2_col else np.nan

if all([rus_col, subj1_col, subj2_col]):
    type_1 = int(((df[rus_col] < rus_mean) & (df[subj1_col] > subj1_mean) & (df[subj2_col] > subj2_mean)).sum())
else:
    type_1 = 0

print("Кол-во студентов Тип_1:", type_1)


### 7) Количество абитуриентов Тип_2 ###

In [ ]:
if all([rus_col, subj1_col, subj2_col]):
    type_2 = int(((df[rus_col] > rus_mean) & (df[subj1_col] < subj1_mean) & (df[subj2_col] < subj2_mean)).sum())
else:
    type_2 = 0

print("Кол-во студентов Тип_2:", type_2)


### 8) Итоговый DataFrame с метаданными ###

In [ ]:
summary_df = pd.DataFrame([
    {
        "Подразделение": "Горно-нефтяной факультет",
        "Уровень подготовки": "Специалитет",
        "Направление подготовки/специальность": "Физические процессы горного или нефтегазового производства",
        "Год": 2025,
        "Количество мест": int(len(df)),
        "Предметы ЕГЭ": "Предмет №1, Предмет №2, Русский язык",
        "Средняя сумма баллов": float(mean_total) if pd.notna(mean_total) else None,
        "Min сумма баллов": float(min_total) if pd.notna(min_total) else None,
        "Max сумма баллов": float(max_total) if pd.notna(max_total) else None,
        "Предмет с высшим средним": top_subject,
        "Кол-во студентов тип_1": int(type_1),
        "Кол-во студентов тип_2": int(type_2),
    }
])

display(summary_df)
